In [4]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np

In [42]:
scores = pd.read_csv('data/historical_round_scores.csv')
pre_tourney = pd.read_csv('pre_tourney_snapshot_2021_08_24.csv')

In [47]:
event_df.query("dg_id == 19195")

,dg_id,player_name,event_name,event_id,fin_text,fin_num,early_out,year,year_event_golfer,event_completed,rounds,mean_score,mean_sg,major,early_out_in_prev_1_events,early_out_in_prev_2_events,early_out_in_prev_5_events,fin_num_in_prev_1_events,fin_num_in_prev_2_events,fin_num_in_prev_5_events,mean_sg_in_prev_1_events,mean_sg_in_prev_2_events,mean_sg_in_prev_5_events
24114,19195,"Rahm, Jon",CareerBuilder Challenge,2,T34,34,0,2017,2017_2_19195,2017-01-22,4,69.75,1.353500,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
24121,19195,"Rahm, Jon",Farmers Insurance Open,4,1,1,0,2017,2017_4_19195,2017-01-29,4,68.75,3.451750,0,0.0,0.0,0.0,34.0,17.0,17.000000,1.353500,0.676750,0.676750
24177,19195,"Rahm, Jon",Waste Management Phoenix Open,3,T16,16,0,2017,2017_3_19195,2017-02-05,4,68.50,1.566750,0,0.0,0.0,0.0,1.0,17.5,11.666667,3.451750,2.402625,1.601750
24108,19195,"Rahm, Jon",AT&T Pebble Beach Pro-Am,5,T5,5,0,2017,2017_5_19195,2017-02-12,4,68.75,3.131750,0,0.0,0.0,0.0,16.0,8.5,12.750000,1.566750,2.509250,1.593000
24189,19195,"Rahm, Jon",World Golf Championships-Mexico Championship,473,T3,3,0,2017,2017_473_19195,2017-03-05,4,68.00,3.049500,0,0.0,0.0,0.0,5.0,10.5,11.200000,3.131750,2.349250,1.900750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24136,19195,"Rahm, Jon",PGA Championship,33,T8,8,0,2021,2021_33_19195,2021-05-23,4,71.75,2.263250,1,0.0,1.0,1.0,34.0,54.5,31.000000,0.835000,0.114250,1.090350
24196,19195,"Rahm, Jon",the Memorial Tournament presented by Nationwide,23,WD,75,1,2021,2021_23_19195,2021-06-06,3,66.00,7.005333,0,0.0,0.0,1.0,8.0,21.0,26.200000,2.263250,1.549125,1.419100
24172,19195,"Rahm, Jon",U.S. Open #2,535,1,1,0,2021,2021_535_19195,2021-06-20,4,69.50,3.770500,1,1.0,1.0,2.0,75.0,41.5,39.400000,7.005333,4.634292,2.394867
24165,19195,"Rahm, Jon",The Open Championship,100,T3,3,0,2021,2021_100_19195,2021-07-18,4,67.25,3.202250,1,0.0,1.0,2.0,1.0,38.0,38.600000,3.770500,5.387917,2.653517


In [8]:
scores['year_event_golfer'] = (scores['year'].astype(str) +
                                      '_' + scores['event_id'].astype(str) +
                                      '_' + scores['dg_id'].astype(str)
                              )

early_outs = ['CUT','DQ','WD', 'W/D', 'MDF',]
early_out_num = 75

scores['early_out'] = np.where(scores.fin_text.isin(early_outs),1,0)

scores['fin_num'] = (np.where(scores.fin_text.isin(early_outs),
                             early_out_num,
                             scores.fin_text.str.replace('T',''))
                    ).astype(int)

# Aggregating round data to an event level
event_df = (scores.groupby(['dg_id','player_name','event_name',
                            'event_id','fin_text','fin_num','early_out',
                            'year','year_event_golfer','event_completed',]).
            agg(rounds=('round','count'),
                mean_score = ('score', 'mean'),
                mean_sg = ('sg_total', 'mean')
               ).reset_index().sort_values('event_completed')
           )

majors = ['Masters Tournament','The Masters','The Masters #2','The Open Championship',
          'U.S. Open','U.S. Open #2','PGA Championship',
         ]

event_df['major'] = np.where(event_df['event_name'].isin(majors), 1, 0)

In [10]:
def create_rolling_agg_features_by_golfer(df, field, n_shift, n_rolling, agg_func):
    """
    """
    df = df.copy()
    
    df['shifted_field'] = (df.groupby('dg_id')[field]
                           .shift(n_shift)
                           .fillna(0)
                          )
    
    return (df.groupby('dg_id')['shifted_field']
            .transform(lambda x: x.rolling(n_rolling, min_periods=1)
                       .agg(agg_func))
           )

fields = 'early_out'

    

In [11]:
metric_lst = [
    {'field': 'early_out','periods': [1, 2, 5], 'agg': 'sum'},
    {'field': 'fin_num', 'periods': [1, 2, 5], 'agg': 'mean'},
    {'field': 'mean_sg', 'periods': [1, 2, 5], 'agg':'mean'},
    ]


In [12]:
for metric in metric_lst:
    field = metric.get('field')
    agg = metric.get('agg')
    for period in metric.get('periods'):
        col = f"{field}_in_prev_{period}_events"
        event_df[col] = create_rolling_agg_features_by_golfer(event_df, field, 1, period, agg)

In [18]:
# fin_by_course = (scores[['course_num', 'course_name', 'dg_id', 'player_name','event_completed','fin_num']]
#                  .drop_duplicates()
#                  .sort_values('event_completed')
#                 )

# fin_by_course['prev_fin_on_course'] = fin_by_course.groupby(['dg_id','course_num'])['fin_num'].shift(1).fillna(40)

### Training

In [19]:
X = event_df[['major', 'early_out_in_prev_1_events',
       'early_out_in_prev_2_events', 'early_out_in_prev_5_events',
       'fin_num_in_prev_1_events', 'fin_num_in_prev_2_events',
       'fin_num_in_prev_5_events', 'mean_sg_in_prev_1_events',
       'mean_sg_in_prev_2_events', 'mean_sg_in_prev_5_events']]

y = event_df['fin_num']

In [40]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [33]:
rf = RandomForestRegressor( n_estimators=100)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42)

In [35]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [37]:
preds = rf.predict(X_test)

In [41]:
mean_absolute_error(y_test, preds)

21.232323845161464